In [1]:
# Full Optimized Code for Bird Classification
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress AVX warnings
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'  # Mitigate OpenMP conflict

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2L
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import numpy as np

In [3]:
# Enhanced Data Augmentation with Mixup and CutMix
class AugmentedGenerator(tf.keras.utils.Sequence):
    def __init__(self, generator, alpha=0.4, cutmix_prob=0.5):
        self.generator = generator
        self.alpha = alpha
        self.cutmix_prob = cutmix_prob

    def __len__(self):
        return len(self.generator)

    def __getitem__(self, index):
        images, labels = self.generator[index]
        
        if np.random.rand() < self.cutmix_prob:
            return self.cutmix(images, labels)
        else:
            return self.mixup(images, labels)
    
    def mixup(self, images, labels):
        indices = np.random.permutation(images.shape[0])
        shuffled_images = images[indices]
        shuffled_labels = labels[indices]
        lam = np.random.beta(self.alpha, self.alpha)
        mixed_images = lam * images + (1 - lam) * shuffled_images
        mixed_labels = lam * labels + (1 - lam) * shuffled_labels
        return mixed_images, mixed_labels

    def cutmix(self, images, labels):
        indices = np.random.permutation(images.shape[0])
        shuffled_images = images[indices]
        shuffled_labels = labels[indices]
        lam = np.random.beta(self.alpha, self.alpha)
        bbx1, bby1, bbx2, bby2 = self.rand_bbox(images.shape, lam)
        images[:, bbx1:bbx2, bby1:bby2, :] = shuffled_images[:, bbx1:bbx2, bby1:bby2, :]
        lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (images.shape[1] * images.shape[2]))
        labels = lam * labels + (1 - lam) * shuffled_labels
        return images, labels

    def rand_bbox(self, size, lam):
        W = size[1]
        H = size[2]
        cut_rat = np.sqrt(1. - lam)
        cut_w = int(W * cut_rat)
        cut_h = int(H * cut_rat)
        cx = np.random.randint(W)
        cy = np.random.randint(H)
        bbx1 = np.clip(cx - cut_w // 2, 0, W)
        bby1 = np.clip(cy - cut_h // 2, 0, H)
        bbx2 = np.clip(cx + cut_w // 2, 0, W)
        bby2 = np.clip(cy + cut_h // 2, 0, H)
        return bbx1, bby1, bbx2, bby2


In [4]:
# Dataset paths (UPDATE THESE TO YOUR LOCAL PATHS)
train_path = 'D:/Dataset/Frog/train'
test_path = 'D:/Dataset/Frog/test'

In [5]:
# Data Generators with Validation Split
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet_v2.preprocess_input,
    rotation_range=60,
    width_shift_range=0.4,
    height_shift_range=0.4,
    shear_range=0.4,
    zoom_range=0.4,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.4, 1.6],
    validation_split=0.2  # 80% training, 20% validation
)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(480, 480),
    batch_size=4,  # Reduced batch size for memory stability
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(480, 480),
    batch_size=4,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

test_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet_v2.preprocess_input
)
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(480, 480),
    batch_size=4,
    class_mode='categorical',
    shuffle=False
)


Found 56 images belonging to 8 classes.
Found 8 images belonging to 8 classes.
Found 16 images belonging to 8 classes.


In [6]:
# Build Enhanced Model Architecture
base_model = EfficientNetV2L(
    include_top=False,
    weights='imagenet',
    input_shape=(480, 480, 3)
)
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dense(2048, activation='swish', kernel_regularizer='l2'),
    Dropout(0.5),
    Dense(1024, activation='swish', kernel_regularizer='l2'),
    Dropout(0.4),
    Dense(8, activation='softmax')  # Ensure this matches your class count
])

In [7]:
# COMPILE THE MODEL BEFORE TRAINING
optimizer = Adam(learning_rate=1e-4, amsgrad=True)
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

In [8]:
# First Training Phase (Frozen Backbone)
history = model.fit(
    AugmentedGenerator(train_generator),
    validation_data=val_generator,
    epochs=50
)

Epoch 1/50
14/14 [==============================] - 98s 5s/step - loss: 31.5477 - accuracy: 0.1250 - val_loss: 31.0646 - val_accuracy: 0.2500
Epoch 2/50
14/14 [==============================] - 58s 4s/step - loss: 31.1322 - accuracy: 0.2321 - val_loss: 30.8323 - val_accuracy: 0.6250
Epoch 3/50
14/14 [==============================] - 60s 4s/step - loss: 30.5680 - accuracy: 0.3393 - val_loss: 30.5127 - val_accuracy: 0.6250
Epoch 4/50
14/14 [==============================] - 60s 4s/step - loss: 30.1191 - accuracy: 0.4643 - val_loss: 30.2294 - val_accuracy: 0.7500
Epoch 5/50
14/14 [==============================] - 64s 5s/step - loss: 30.1271 - accuracy: 0.2679 - val_loss: 30.0179 - val_accuracy: 0.6250
Epoch 6/50
14/14 [==============================] - 67s 5s/step - loss: 29.6828 - accuracy: 0.5000 - val_loss: 29.7162 - val_accuracy: 0.6250
Epoch 7/50
14/14 [==============================] - 66s 5s/step - loss: 29.3823 - accuracy: 0.5893 - val_loss: 29.4359 - val_accuracy: 0.5000
Epoch 

In [9]:
model.save('D:/Model_Main/Xception_net_client_Frog_cnn_02.h5')

In [10]:
# Fine-tuning Phase
base_model.trainable = True
for layer in base_model.layers[:300]:
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

history_fine = model.fit(
    AugmentedGenerator(train_generator),
    validation_data=val_generator,
    initial_epoch=history.epoch[-1],
    epochs=history.epoch[-1] + 30,
    
)

Epoch 50/79
14/14 [==============================] - 186s 10s/step - loss: 19.5929 - accuracy: 0.6429 - val_loss: 19.1062 - val_accuracy: 0.8750
Epoch 51/79
14/14 [==============================] - 136s 10s/step - loss: 19.7797 - accuracy: 0.5714 - val_loss: 19.4024 - val_accuracy: 0.7500
Epoch 52/79
14/14 [==============================] - 139s 10s/step - loss: 19.5377 - accuracy: 0.7321 - val_loss: 19.1371 - val_accuracy: 0.7500
Epoch 53/79
14/14 [==============================] - 140s 10s/step - loss: 19.5365 - accuracy: 0.6607 - val_loss: 18.8321 - val_accuracy: 1.0000
Epoch 54/79
14/14 [==============================] - 136s 10s/step - loss: 19.6349 - accuracy: 0.5714 - val_loss: 18.9670 - val_accuracy: 0.7500
Epoch 55/79
14/14 [==============================] - 137s 10s/step - loss: 19.3705 - accuracy: 0.6786 - val_loss: 18.9314 - val_accuracy: 0.8750
Epoch 56/79
14/14 [==============================] - 139s 10s/step - loss: 19.3883 - accuracy: 0.8036 - val_loss: 19.1161 - val_ac

In [11]:
model.save('D:/Model_Main/Xception_net_client_Frog_cnn_tunned_02.h5')

In [13]:
# Evaluate the model
eval_results = model.evaluate(test_generator)
print(f"Test Loss: {eval_results[0]:.4f} | Test Accuracy: {eval_results[1]:.4f}")

4/4 [==============================] - 13s 3s/step - loss: 21.3749 - accuracy: 0.1250 
Test Loss: 21.3749 | Test Accuracy: 0.1250


In [37]:
from tensorflow.keras.models import load_model


In [38]:


# Load the trained model
model = load_model('D:/Model_Main/Xception_net_client_Frog_cnn_tunned_02.h5')



In [63]:
class_labels = {
    0: 'Bull',
    1: 'Charles',
    2: 'Shompen',
    3: 'Wart',
    4: 'Chorus',
    5: 'Common',
    6: 'Copper',
    7: 'Red'
}

In [55]:
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input

In [71]:
# Load and preprocess the image
image_path = 'D:/Less_data_test/frog14.jpg'
test_image = load_img(image_path, target_size=(480, 480))  # Match model input size
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image = preprocess_input(test_image)  # Use EfficientNetV2 preprocessing

# Predict the class
result = model.predict(test_image)
predicted_class = np.argmax(result)

# Get the predicted label
prediction = class_labels[predicted_class]

print(f'The predicted class is: {prediction}')


1/1 [==============================] - 1s 1s/step
The predicted class is: Wart


### 